In [1]:
!nvidia-smi

Tue Jul 23 13:02:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:02:00.0 Off |                   On |
| N/A   28C    P0              25W / 165W |     50MiB / 24576MiB |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [2]:
from dataclasses import dataclass
from functools import partial
from math import floor 
from typing import Callable, Tuple, Any

import jax
from jax import debug
import jax.numpy as jnp
import flax.linen as nn
import optax
from chex import ArrayTree
from qdax.core.containers.repertoire import Repertoire
from qdax.types import Descriptor, ExtraScores, Fitness, Genotype, RNGKey
from qdax.environments.base_wrappers import QDEnv
from qdax.core.neuroevolution.buffers.buffer import QDTransition, QDMCTransition
#from qdax.core.neuroevolution.buffers.trajectory_buffer import TrajectoryBuffer
import flashbax as fbx
import chex
from rein_related import *

from qdax.core.emitters.emitter import Emitter, EmitterState

In [3]:
import os

os.environ['MPLCONFIGDIR'] = '/tmp/matplotlib'
os.environ['WANDB_CACHE_DIR'] = '/tmp/wandb_cache'
os.environ['JAX_LOG_COMPILATION'] = '1'

import logging
import time
from dataclasses import dataclass
from functools import partial
from math import floor
from typing import Any, Dict, Tuple, List, Callable
import pickle
from flax import serialization
#logging.basicConfig(level=logging.DEBUG)
import hydra
from omegaconf import OmegaConf, DictConfig
import jax
import jax.numpy as jnp
from hydra.core.config_store import ConfigStore
from qdax.core.map_elites import MAPElites
from qdax.types import RNGKey, Genotype
from qdax.utils.sampling import sampling 
from qdax.core.containers.mapelites_repertoire import compute_cvt_centroids, MapElitesRepertoire
from qdax.core.neuroevolution.networks.networks import MLPMCPG
from qdax.core.emitters.me_mcpg_emitter import MEMCPGConfig, MEMCPGEmitter
#from qdax.core.emitters.rein_emitter_advanced import REINaiveConfig, REINaiveEmitter
from qdax.core.neuroevolution.buffers.buffer import QDTransition, TestTransition
from qdax.environments import behavior_descriptor_extractor
from qdax.tasks.brax_envs import reset_based_scoring_function_brax_envs as scoring_function
from utils import Config, get_env
from qdax.core.emitters.mutation_operators import isoline_variation
import wandb
from qdax.utils.metrics import CSVLogger, default_qd_metrics
from qdax.utils.plotting import plot_map_elites_results, plot_2d_map_elites_repertoire
import matplotlib.pyplot as plt
from set_up_brax import get_reward_offset_brax
from qdax import environments_v1, environments

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import jax.numpy as jnp  # Assuming you are using jax.numpy as jnp

In [5]:
def get_env(env_name):
    if env_name == "hopper_uni":
        episode_length = 1000
        
        env = environments_v1.create(env_name, episode_length=episode_length)
    elif env_name == "halfcheetah_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)
        
    elif env_name == "walker2d_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)	
    elif env_name == "ant_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=True)
    elif env_name == "humanoid_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, exclude_current_positions_from_observation=True)	
    '''
    elif env_name == "ant_omni":
        episode_length = 250
        max_bd = 30.

        env = environments.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=False)	
    elif env_name == "humanoid_uni":
        episode_length = 1000
        max_bd = 1.

        env = environments.create(env_name, episode_length=episode_length)	
    else:
        ValueError(f"Environment {env_name} not supported.")
    '''
    return env

In [6]:
@dataclass
class Config:
    """Configuration from this experiment script
    """
    # Env config
    #alg_name: str
    seed: int
    env_name: str
    episode_length: int
    policy_hidden_layer_sizes: Tuple[int, ...]   
    # ME config
    num_evaluations: int
    num_iterations: int
    no_agents: int
    num_samples: int
    fixed_init_state: bool
    discard_dead: bool
    # Emitter config
    iso_sigma: float
    line_sigma: float
    #crossover_percentage: float
    # Grid config 
    grid_shape: Tuple[int, ...]
    num_init_cvt_samples: int
    num_centroids: int
    # Log config
    log_period: int
    store_repertoire: bool
    store_repertoire_log_period: int
    
    # REINFORCE Parameters
    proportion_mutation_ga : float
    buffer_sample_batch_size : int
    buffer_add_batch_size: int
    adam_optimizer: bool
    learning_rate: float
    discount_rate: float
    #buffer_size: int
    clip_param: float
    no_epochs: int

In [7]:
no_epochs = [4, 8, 16]

envs = ["hopper_uni", "walker2d_uni", "ant_uni"]

os.makedirs("grad_steps_experiments/reps=16/lr=3e-5", exist_ok=True)


for env_ in envs:
    
    env_dir = f"grad_steps_experiments/reps=16/lr=3e-5/{env_}"
    os.makedirs(env_dir, exist_ok=True)
    
    for no_epoch in no_epochs:
        config = Config(
            seed=0,
            env_name=env_,
            episode_length=1000,
            policy_hidden_layer_sizes=[128, 128],
            num_evaluations=0,
            num_iterations=4000,
            num_samples=8,
            no_agents=512,
            fixed_init_state=False,
            discard_dead=False,
            grid_shape=[50, 50],
            num_init_cvt_samples=50000,
            num_centroids=1024,
            log_period=400,
            store_repertoire=True,
            store_repertoire_log_period=800,
            iso_sigma=0.005,
            line_sigma=0.05,
            proportion_mutation_ga=0.5,
            buffer_sample_batch_size=8,
            buffer_add_batch_size=512,
            no_epochs=no_epoch,
            #buffer_size=64000,
            adam_optimizer=True,
            learning_rate=3e-4,
            discount_rate=0.99,
            clip_param=0.2
        )

        random_key = jax.random.PRNGKey(config.seed)

        # Init environment
        env = get_env(env_)
        reset_fn = jax.jit(env.reset)

        # Compute the centroids
        centroids, random_key = compute_cvt_centroids(
            num_descriptors=env.behavior_descriptor_length,
            num_init_cvt_samples=config.num_init_cvt_samples,
            num_centroids=config.num_centroids,
            minval=0,
            maxval=1,
            random_key=random_key,
        )
        # Init policy network
        policy_layer_sizes = config.policy_hidden_layer_sizes #+ (env.action_size,)
        print(policy_layer_sizes)




        policy_network = MLPMCPG(
            hidden_layers_size=policy_layer_sizes,
            action_size=env.action_size,
            activation=jax.nn.tanh,
            hidden_init=jax.nn.initializers.orthogonal(scale=jnp.sqrt(2)),
            mean_init=jax.nn.initializers.orthogonal(scale=0.01),
        )


            
        '''
        policy_network = MLPMCPG(
            hidden_layers_size=policy_layer_sizes,
            action_size=env.action_size,
            activation=jax.nn.tanh,
            hidden_init=jax.nn.initializers.variance_scaling(scale=jnp.sqrt(2), mode='fan_in', distribution='uniform'),
            mean_init=jax.nn.initializers.variance_scaling(scale=0.02*jnp.sqrt(2), mode='fan_in', distribution='uniform'),
        )
        '''


        '''
        policy_network = MLPMCPG(
            hidden_layers_size=policy_layer_sizes,
            action_size=env.action_size,
            activation=jax.nn.tanh,
            hidden_init=jax.nn.initializers.lecun_uniform(),
            mean_init=jax.nn.initializers.lecun_uniform(),
        )
        '''



        # Init population of controllers

        # maybe consider adding two random keys for each policy
        random_key, subkey = jax.random.split(random_key)
        keys = jax.random.split(subkey, num=config.no_agents)
        #split_keys = jax.vmap(lambda k: jax.random.split(k, 2))(keys)
        #keys1, keys2 = split_keys[:, 0], split_keys[:, 1]
        fake_batch_obs = jnp.zeros(shape=(config.no_agents, env.observation_size))
        init_params = jax.vmap(policy_network.init)(keys, fake_batch_obs)

        param_count = sum(x[0].size for x in jax.tree_util.tree_leaves(init_params))
        print("Number of parameters in policy_network: ", param_count)

        # Define the fonction to play a step with the policy in the environment
        @jax.jit
        def play_step_fn(env_state, policy_params, random_key):
            #random_key, subkey = jax.random.split(random_key)
            actions, logp = policy_network.apply(policy_params, env_state.obs)
            #logp = policy_network.apply(policy_params, env_state.obs, actions, method=policy_network.logp)
            state_desc = env_state.info["state_descriptor"]
            next_state = env.step(env_state, actions)

            transition = TestTransition(
                obs=env_state.obs,
                #next_obs=next_state.obs,
                rewards=next_state.reward,
                dones=next_state.done,
                truncations=next_state.info["truncation"],
                actions=actions,
                state_desc=state_desc,
                #next_state_desc=next_state.info["state_descriptor"],
                logp=logp,
                #desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
                #desc_prime=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
            )

            return next_state, policy_params, random_key, transition


        # Prepare the scoring function
        bd_extraction_fn = behavior_descriptor_extractor['ant_uni']
        scoring_fn = partial(
            scoring_function,
            episode_length=env.episode_length,
            play_reset_fn=reset_fn,
            play_step_fn=play_step_fn,
            behavior_descriptor_extractor=bd_extraction_fn,
        )
        #reward_offset = get_reward_offset_brax(env, config.env_name)
        #print(f"Reward offset: {reward_offset}")

        me_scoring_fn = partial(
        sampling,
        scoring_fn=scoring_fn,
        num_samples=config.num_samples,
        )

        reward_offset = 0

        # Define a metrics function
        metrics_function = partial(
            default_qd_metrics,
            qd_offset=reward_offset * env.episode_length,
        )

        # Define the PG-emitter config

        me_mcpg_config = MEMCPGConfig(
            proportion_mutation_ga=config.proportion_mutation_ga,
            no_agents=config.no_agents,
            buffer_sample_batch_size=config.buffer_sample_batch_size,
            buffer_add_batch_size=config.buffer_add_batch_size,
            #batch_size=config.batch_size,
            #mini_batch_size=config.mini_batch_size,
            no_epochs=config.no_epochs,
            #buffer_size=config.buffer_size,
            learning_rate=config.learning_rate,
            adam_optimizer=config.adam_optimizer,
            clip_param=config.clip_param,
        )

        variation_fn = partial(
            isoline_variation, iso_sigma=config.iso_sigma, line_sigma=config.line_sigma
        )

        me_mcpg_emitter = MEMCPGEmitter(
            config=me_mcpg_config,
            policy_network=policy_network,
            env=env,
            variation_fn=variation_fn,
            )

        '''
        rein_emitter = REINaiveEmitter(
            config=rein_emitter_config,
            policy_network=policy_network,
            env=env,
            )
        '''
        '''
        me_scoring_fn = partial(
            sampling,
            scoring_fn=scoring_fn,
            num_samples=config.num_samples,
        )
        '''

        # Instantiate MAP Elites
        map_elites = MAPElites(
            scoring_function=scoring_fn,
            emitter=me_mcpg_emitter,
            metrics_function=metrics_function,
        )

        fitnesses, descriptors, extra_scores, random_key = scoring_fn(
            init_params, random_key
        )
        
        repertoire = MapElitesRepertoire.init(
            genotypes=init_params,
            fitnesses=fitnesses,
            descriptors=descriptors,
            centroids=centroids,
            extra_scores=extra_scores,
        )
        

        emitter_state, random_key = me_mcpg_emitter.init(
            random_key=random_key,
            repertoire=repertoire,
            genotypes=init_params,
            fitnesses=fitnesses,
            descriptors=descriptors,
            extra_scores=extra_scores,
        )
        
        emitter_state = me_mcpg_emitter.state_update(
            emitter_state=emitter_state,
            repertoire=repertoire,
            genotypes=init_params,
            fitnesses=fitnesses,
            descriptors=descriptors,
            extra_scores={**extra_scores}#, **extra_info},
        )
        
        emitter_state = emitter_state.emitter_states[0]
        
        


        returns = []
        old_params = init_params
        random_key = jax.random.PRNGKey(0)
        for _ in range(250):
            random_keys = jax.random.split(random_key, config.no_agents)
            new_params = me_mcpg_emitter.emitters[0].emit_mcpg(emitter_state, old_params, random_keys)
            fitnesses, descriptors, extra_scores, random_key = scoring_fn(
                new_params, random_key
            )
            emitter_state = me_mcpg_emitter.emitters[0].state_update(
                emitter_state=emitter_state,
                repertoire=repertoire,
                genotypes=new_params,
                fitnesses=fitnesses,
                descriptors=descriptors,
                extra_scores=extra_scores,
            )
            old_params = new_params
            print(f"mean fitness: {fitnesses.mean()}")
            returns.append(fitnesses)
            
        returns = jnp.array(returns)  # Assuming 'returns' is already defined as a 2D array

        # Determine the overall min and max fitness values for setting y-axis limits
        ymin = returns.min()
        ymax = returns.max()

        # Plotting
        fig, axs = plt.subplots(16, 32, figsize=(40, 80))  # Adjust the subplot grid to 16x32
        for i in range(512):  # Loop through 512 plots
            ax = axs[i // 32, i % 32]  # This assumes a 16x32 grid of subplots
            ax.plot(returns[:, i])
            ax.set_title(f"Policy {i+1}", fontsize=8)
            ax.set_xlabel('Steps', fontsize=6)
            ax.set_ylabel('Returns', fontsize=6)
            ax.set_ylim([ymin, ymax])  # Set the same y-axis limits for all subplots
            ax.tick_params(axis='both', which='major', labelsize=6)

        plt.tight_layout()
        # Save each plot in the specified directory with the correct filename
        plot_filename = f"{env_dir}/no_epoch={no_epoch}.png"
        plt.savefig(plot_filename)
        plt.close(fig)  # Close the plot to free up memory

[128, 128]
Number of parameters in policy_network:  18566


/tmp/ipykernel_3278128/2864921634.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 88.00109100341797
mean fitness: 126.16275787353516
mean fitness: 152.96363830566406
mean fitness: 157.14422607421875
mean fitness: 194.07009887695312
mean fitness: 201.42161560058594
mean fitness: 216.52151489257812
mean fitness: 213.85581970214844
mean fitness: 232.61302185058594
mean fitness: 241.1270751953125
mean fitness: 248.13601684570312
mean fitness: 275.18389892578125
mean fitness: 280.79345703125
mean fitness: 311.85418701171875
mean fitness: 343.25531005859375
mean fitness: 358.8013610839844
mean fitness: 380.5671081542969
mean fitness: 386.6127014160156
mean fitness: 407.6954040527344
mean fitness: 442.1996154785156
mean fitness: 461.1076965332031
mean fitness: 437.84716796875
mean fitness: 477.02593994140625
mean fitness: 461.0866394042969
mean fitness: 471.48492431640625
mean fitness: 493.27911376953125
mean fitness: 516.0028076171875
mean fitness: 505.21722412109375
mean fitness: 521.037841796875
mean fitness: 519.4825439453125
mean fitness: 537.69250488281

/tmp/ipykernel_3278128/2864921634.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 84.36808776855469
mean fitness: 120.8125
mean fitness: 232.6066436767578
mean fitness: 233.5299072265625
mean fitness: 237.28433227539062
mean fitness: 317.8188781738281
mean fitness: 372.75335693359375
mean fitness: 364.16424560546875
mean fitness: 422.94903564453125
mean fitness: 423.3612365722656
mean fitness: 430.635009765625
mean fitness: 463.67791748046875
mean fitness: 467.011962890625
mean fitness: 475.2060546875
mean fitness: 505.2001953125
mean fitness: 540.0714111328125
mean fitness: 568.740478515625
mean fitness: 558.6610107421875
mean fitness: 574.806396484375
mean fitness: 584.578369140625
mean fitness: 589.9857177734375
mean fitness: 594.3070068359375
mean fitness: 619.2737426757812
mean fitness: 611.7503051757812
mean fitness: 622.0484008789062
mean fitness: 576.344482421875
mean fitness: 586.1343383789062
mean fitness: 577.454345703125
mean fitness: 561.5292358398438
mean fitness: 544.210693359375
mean fitness: 565.716552734375
mean fitness: 538.365478515

...

mean fitness: 966.617431640625
mean fitness: 983.8422241210938
mean fitness: 977.2520751953125
mean fitness: 989.9725952148438
mean fitness: 977.7943115234375
mean fitness: 984.1904296875
mean fitness: 976.1355590820312
mean fitness: 979.7142333984375
mean fitness: 978.1154174804688
mean fitness: 980.423828125
mean fitness: 980.0010375976562
mean fitness: 981.5294189453125
mean fitness: 981.8724365234375
mean fitness: 982.2446899414062
mean fitness: 991.7091064453125
mean fitness: 993.8939819335938
mean fitness: 1000.0147094726562
mean fitness: 1006.232177734375
mean fitness: 1014.67578125
mean fitness: 1018.2395629882812
mean fitness: 1025.35595703125
mean fitness: 1061.5712890625
mean fitness: 1072.3526611328125
mean fitness: 1099.1622314453125
mean fitness: 1113.1693115234375
mean fitness: 1139.909423828125
mean fitness: 1135.458251953125
mean fitness: 1140.7750244140625
mean fitness: 1161.0802001953125
mean fitness: 1157.267822265625
mean fitness: 1165.146728515625
mean fitness: 11

/tmp/ipykernel_3278128/2864921634.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 968.7622680664062
mean fitness: 976.666015625
mean fitness: 964.4151611328125
mean fitness: 972.84814453125
mean fitness: 968.4072265625
mean fitness: 978.06494140625
mean fitness: 978.6226196289062
mean fitness: 983.582763671875
mean fitness: 981.62548828125
mean fitness: 983.4686889648438
mean fitness: 993.6744384765625
mean fitness: 1007.9906616210938
mean fitness: 1017.7732543945312
mean fitness: 1045.828857421875
mean fitness: 1062.037109375
mean fitness: 1089.7890625
mean fitness: 1106.866455078125
mean fitness: 1139.490966796875
mean fitness: 1167.829345703125
mean fitness: 1179.845458984375
mean fitness: 1202.9178466796875
mean fitness: 1213.1182861328125
mean fitness: 1211.3812255859375
mean fitness: 1254.078125
mean fitness: 1253.643310546875
mean fitness: 1254.7822265625
mean fitness: 1275.989990234375
mean fitness: 1274.710693359375
mean fitness: 1294.74609375
mean fitness: 1305.9591064453125
mean fitness: 1305.427978515625
mean fitness: 1329.6104736328125
mea

/tmp/ipykernel_3278128/2864921634.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 923.4525146484375
mean fitness: 956.4696044921875
mean fitness: 946.1681518554688
mean fitness: 981.7708740234375
mean fitness: 1011.73779296875
mean fitness: 974.197998046875
mean fitness: 1062.598876953125
mean fitness: 1010.7026977539062
mean fitness: 1082.801025390625
mean fitness: 1064.724853515625
mean fitness: 1095.89404296875
mean fitness: 1117.403076171875
mean fitness: 1142.58935546875
mean fitness: 1167.7359619140625
mean fitness: 1196.540283203125
mean fitness: 1206.8779296875
mean fitness: 1229.016357421875
mean fitness: 1254.635986328125
mean fitness: 1261.0255126953125
mean fitness: 1272.254638671875
mean fitness: 1311.1416015625
mean fitness: 1301.6014404296875
mean fitness: 1314.8636474609375
mean fitness: 1323.2337646484375
mean fitness: 1335.0606689453125
mean fitness: 1351.068359375
mean fitness: 1349.868896484375
mean fitness: 1361.9012451171875
mean fitness: 1377.127685546875
mean fitness: 1379.990478515625
mean fitness: 1396.44482421875
mean fitness